# 第10章 TensorFlow高层封装
虽然原生态的TensorFlow API可以很灵活地支持不同的神经网络结构，但是其代码相对冗长，写起来比较麻烦。**为了让用户更加方便快捷地实现常用的神经网络结构，不同的组织和个人为TensorFlow提供了多种高级封装**。第6章已经简单介绍了TensorFlow的高层封装，并使用它实现了CNN，这一章将更详细地介绍几种最常用的TensorFlow高层封装。

## 10.1 TensorFlow高层封装总览
**目前比较主流的TensorFlow高层封装主要有四个，分别是TensorFlow-SLim、TFLearn、Keras和Estimator。**

**1. TensorFLow-Slim**

TensorFLow-Slim是Google官方给出的相对较早的TensorFlow高层封装，Google通过TensorFLow-Slim开源了一些已经训练好的图像分析的模型，所以目前在图像识别问题中TensorFLow-Slim仍被较多地使用，下面给出一个简单的样例，介绍了如何使用TensorFLow-Slim在MNIST数据集上实现LeNet5模型：

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data


# 通过TensorFlow-Slim来定义LeNet-5的网络结构。
def lenet5(inputs):
    # 将输入转化为一个4维数组，第一维是batch大小，后面三维表示一张图片
    inputs = tf.reshape(inputs, [-1, 28, 28, 1])
    
    # 定义第一卷积层。可以看出，TensorFLow-Slim定义的网络结构并不需要用户去关心如何声明和初始化变量，
    # 而只需要定义网络结构即可。
    net = slim.conv2d(inputs, 32, [5, 5], padding='SAME', scope='layer1-conv')
    net = slim.max_pool2d(net, 2, stride=2, scope='layer2-max-pool')
    
    # 定义第二卷积层
    net = slim.conv2d(net, 64, [5, 5], padding='SAME', scope='layer3-conv')
    net = slim.max_pool2d(net, 2, stride=2, scope='layer4-max-pool')
    
    # 直接使用TensorFLow-Slim封装好的flatten函数将4维矩阵将为2维，用户不需计算通过卷积后的矩阵大小。
    net = slim.flatten(net, scope='flatten')
    
    # 通过TensorFLow-Slim定义全连接层，500是节点数。
    net = slim.fully_connected(net, 500, scope='layer5')
    net = slim.fully_connected(net, 10, scope='output')
    
    return net


# 通过TensorFLow-Slim定义网络结构，并使用之前章节中给出的方式训练定义好的模型。
def train(mnist):
    # 1. 定义网络输入输出
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
    # 2. 定义前向传播、损失函数、反向传播
    y = lenet5(x)

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    loss = tf.reduce_mean(cross_entropy)

    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    # 其他准备，feed、accuracy
    feed_valid = {x:mnist.validation.images, y_:mnist.validation.labels}
    feed_test = {x:mnist.test.images, y_:mnist.test.labels}
    correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    # 3. 建立会话，训练
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(5000):
            xs, ys = mnist.train.next_batch(100)
            _, loss_value = sess.run([train_op, loss], feed_dict={x: xs, y_: ys})

            if i % 1000 == 0 or i == mnist.validation.images.shape[0] - 1:
                acc_valid = sess.run(accuracy, feed_dict=feed_valid)
                print("After %d training step(s), loss on training batch is %g, accuracy on validation is %f." % (i, loss_value, acc_valid))
                
        acc_test = sess.run(accuracy, feed_dict=feed_test)
        print('Final test accuracy is %f.' % acc_test)
                
                
def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 0 training step(s), loss on training batch is 2.30333, accuracy on validation is 0.065600.
After 1000 training step(s), loss on training batch is 0.426152, accuracy on validation is 0.851800.
After 2000 training step(s), loss on traini

从以上代码可以看出，**TensorFlow-Slim主要的作用是使模型定义更加简洁，基本上每一层网络可以通过一句话来实现。除了对单层网络结构，TensorFlow-Slim 还对数据预处理、损失函数、学习过程、测试过程等都提供了高层封装**不过因为TensorFlow-Slim的这些封装使用得并不广泛，所以本书不做详细介绍，感兴趣的读者可以参考GitHub上TensorFlow-Slim的[代码库](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)。

**TensorFlow-Slim最特别的一个地方是它对一些标准的神经网络模型进行了封装，比如VGG、Inception以及ResNet**，而且Google开源的训练好的图像分类模型基本都是通过TensorFlow-Slim实现的。第6章介绍迁移学习时已经使用过通过TensorFlow-Slim定义的Inception-v3模型了。更加详细的、通过TensorFlow-Slim开源的训练好的模型列表可以在GitHub上[找到](https://github.com/tensorflow/models/tree/master/research/slim/nets)。

**2. TFLearn**

与TensorFlow-Slim相比，TFLearn是一个更加简洁的TensorFlow高层封装。通过TFLearn可以更加容易地完成模型定义、模型训练以及模型评测地全过程。TFLearn没有集成在TensorFlow地安装包中，故需要单独安装：`pip install tflearn`。以下代码展示了如何使用TFLearn在MNIST数据集上实现LeNet5模型：

In [1]:
import tflearn
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

import tflearn.datasets.mnist as mnist

# 读取mnist数据集
trainX, trainY, testX, testY = mnist.load_data("../../datasets/MNIST_data", one_hot=True)

# 将图像数据reshape成CNN输入的格式
trainX = trainX.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

# 构建神经网络，这个过程和tensorFlow-Slim比较类似
# input_data定义了一个placeholder来接入输入数据
net = input_data(shape=[None, 28, 28, 1], name='input')
# 通过TFLearn封装好的API定义一个深度为5，过滤器为5*5，激活函数为ReLU的卷积层
net = conv_2d(net, nb_filter=32, filter_size=5, activation='relu')
# 定义了一个过滤器为2*2的最大池化层
net = max_pool_2d(net, kernel_size=2)
# 类似地定义其他地网络结构
net = conv_2d(net, 64, 5, activation='relu')
net = max_pool_2d(net, 2)
net = fully_connected(net, 500, activation='relu')
net = fully_connected(net, 10, activation='softmax')

# 使用TFLearn封装好的函数定义学习任务。指定优化器为sgd，学习率为0.01，损失函数为交叉熵
net = regression(net, optimizer='sgd', learning_rate=0.01, \
                 loss='categorical_crossentropy')

# 通过定义地网络结构训练模型，并在指定的验证集上验证模型效果。
# TFLearn将模型的训练过程封装到一个类中，这样可以减少非常多的冗余代码
model = tflearn.DNN(net, tensorboard_verbose=0)

model.fit(trainX, trainY, n_epoch=20, validation_set=([testX, testY]), show_metric=True)

Training Step: 17199  | total loss: 0.02664 | time: 7.026s
| SGD | epoch: 020 | loss: 0.02664 - acc: 0.9917 -- iter: 54976/55000
Training Step: 17200  | total loss: 0.02501 | time: 8.214s
| SGD | epoch: 020 | loss: 0.02501 - acc: 0.9925 | val_loss: 0.03196 - val_acc: 0.9887 -- iter: 55000/55000
--


从以上代码可以看出，**使用TFLearn训练神经网络的流程也是一样的：先定义神经网络的结构，再使用训练数据来训练、模型**。

**TFLearn vs. 原生态TensorFlow：**

- TFLearn不仅使神经网络结构定义更加简洁，还将模型训练的过程也进行了封装。另外，在定义神经网络的前向传播过程之后， TFLearn可以**通过regression函数来指定损失函数和优化方法**。更方便的是，**不仅TFLearn能很好地封装模型定义，tflearn.DNN也能很好地封装模型训练的过程**。通过fit函数可以指定训练中使用的数据和训练的轮数。这样避免了大量的冗余代码。

**TFLearn vs. Keras&Esitimator:**
- 相同点：封装的方式上基本一致，主要也是针对模型定义和模型训练两个部分；
- 不同点：Keras和Estimator都己经加入了TensorFlow代码库，而且它们是使用最为广泛的TensorFlow高层封装。

限于篇幅，本书不在介绍TFLearn的复杂方法的使用，感兴趣的读者可以参考[TFLearn官网](http://tflearn.org/)上的相关内容。